# 1 Processing

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [ ]:
#clearml mac
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=QO1VBBX9J2S2VYILQTGI
# %env CLEARML_API_SECRET_KEY=ERuc1S6o5SirQGugvYXDFjH9b9aNi0u8S3rpALzXMa8YPSLDMW

# clearml win
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
%env CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn

In [ ]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
# read config file
import yaml

def load_config(config_file):
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    return config

config = load_config('seg_yolo_config_4.yaml')

print(config)

# project
project = config['project']
task_name = config['task_name']
exist_ok = config['exist_ok']

# dataset
# data = config['data_mac']
data = config['data_win']
test_data = config['test_data']
imgsz = config['imgsz']

# model
model = config['model']
task = config['task']
mode = config['mode']
# device = config['device_mac']
device = config['device_win']

# Training parameters
epochs = config['epochs']
batch = config['batch']
lr0 = config['lr0']
cos_lr = config['cos_lr']
weight_decay = config['weight_decay']
dropout = config['dropout']
optimizer = config['optimizer']
momentum = config['momentum']
seed = config['seed']

# Output configuration
# save = config['save']
# save_period = config['save_period']
plots = config['plots']
test_output = config['test_output']


In [ ]:
import pandas as pd
import math

Dataset_version = "v2"
train_amount = 1113
batch_size = batch
total_epoch =  epochs
base_lr = lr0

test_record_csv = "/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/Test_seg_yolo_record.csv"

# Save in csv
new_row = [
    task_name,
    Dataset_version,
    model,
    train_amount,
    batch_size,
    total_epoch,
    base_lr,
    "none"
]

df = pd.read_csv(test_record_csv, header=None)
df.loc[len(df)] = new_row

df.to_csv(test_record_csv, header=False, index=False)

In [ ]:
import clearml
from clearml import Task

#Clear ML Initialization
cl_task = Task.init(project_name=project,task_name=task_name)
logger = cl_task.get_logger()

# 2 Training

In [ ]:
from ultralytics import YOLO

ini_model = "/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/yolov8m-seg.pt"
model = YOLO(ini_model)

results = model.train(project=project, name=task_name, exist_ok=exist_ok, data=data, imgsz=imgsz, task=task, device=device, epochs=epochs, batch=batch, lr0=lr0, cos_lr=cos_lr, weight_decay=weight_decay, dropout=dropout, optimizer=optimizer, momentum=momentum, seed=seed, plots=plots)

cl_task.close()

# 3 testing

## 3.1 path

In [4]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.45 🚀 Python-3.11.5 torch-2.1.1 CPU (Apple M1 Max)
Setup complete ✅ (10 CPUs, 32.0 GB RAM, 868.5/1858.2 GB disk)


In [30]:
from ultralytics import YOLO
import cv2
import os

model_path = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/weights/best.pt"

image_folder = "/Users/holmes/Documents/UNI-Bamberg/Arbeiten/RAW-DATA/FARO_pano_pc/test"

save_path = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/tests/test_more/resized"



## 3.2 resize

In [31]:
# Resize images
resized_image_folder = os.path.join(image_folder, "resized")
os.makedirs(resized_image_folder, exist_ok=True)

for file in os.listdir(image_folder):
    if file.endswith(('.jpg', '.JPG', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (640, 640))
        resized_img_path = os.path.join(resized_image_folder, os.path.splitext(file)[0] + "_resize" + os.path.splitext(file)[1])
        cv2.imwrite(resized_img_path, img_resized)
image_folder = resized_image_folder


## 3.3 test

In [9]:
model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    results = model.predict(img_path)
    colors = [random.choices(range(256), k=3) for _ in classes_ids]

    img = cv2.imread(img_path)

    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            # box contains prediction information
            confidence = box.conf[0]
            if confidence > 0.4:
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 0, 255), 2)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    unique_save_path = os.path.join(save_path, os.path.basename(img_path))
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")



image 1/1 /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/tests/resized/Model_3_0021_jpeg.rf.056041b53706e5d9cea426153674d61d_resize.jpg: 640x640 6 side_beams, 440.6ms
Speed: 2.4ms preprocess, 440.6ms inference, 13.9ms postprocess per image at shape (1, 3, 640, 640)
Image saved: /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/tests/Model_3_0021_jpeg.rf.056041b53706e5d9cea426153674d61d_resize.jpg


In [32]:
from ultralytics import YOLO
import random
import cv2
import numpy as np

model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    results = model.predict(img_path)
    img = cv2.imread(img_path)

    overlay = np.zeros_like(img)

    for result in results:
        if result.masks is not None:
            for mask, box in zip(result.masks.xy, result.boxes):
                random_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                
                points = np.array([mask], dtype=np.int32) 
                cv2.fillPoly(overlay, points, random_color) 
                
                confidence = box.conf[0]
                if confidence > 0.4:
                    xyxy = box.xyxy[0].cpu().numpy().astype(int)
                    cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), random_color, 2)
                    cv2.putText(img, confidence_text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, random_color, 2)

    cv2.addWeighted(overlay, 0.4, img, 0.6, 0, img) 

    unique_save_path = os.path.join(save_path, os.path.basename(img_path))
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")



image 1/1 /Users/holmes/Documents/UNI-Bamberg/Arbeiten/RAW-DATA/FARO_pano_pc/test/resized/1_5_resize.png: 640x640 1 main_beam, 3 side_beams, 435.1ms
Speed: 1.3ms preprocess, 435.1ms inference, 4.3ms postprocess per image at shape (1, 3, 640, 640)
Image saved: /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/tests/test_more/resized/1_5_resize.png

image 1/1 /Users/holmes/Documents/UNI-Bamberg/Arbeiten/RAW-DATA/FARO_pano_pc/test/resized/2_3_resize.png: 640x640 1 main_beam, 4 side_beams, 407.5ms
Speed: 1.1ms preprocess, 407.5ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)
Image saved: /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/tests/test_more/resized/2_3_resize.png

image 1/1 /Users/holmes/Documents/UNI-Bamberg/Arbeiten/RAW-DATA/FARO_pano_pc/test/resized/Bildexport_3_intensity_resize.jpg: 640x

In [23]:
import json
import os
import random
import cv2
import numpy as np
from ultralytics import YOLO

def mask_to_polygons(mask):
    # Ensure mask is a 2D numpy array of type uint8
    if mask.ndim > 2:
        mask = mask[:, :, 0]  # Assuming mask might be multi-channel
    if mask.dtype != np.uint8:
        mask = mask.astype(np.uint8)  # Convert mask to uint8
    
    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    polygons = [contour.flatten().tolist() for contour in contours if len(contour) > 0]
    return polygons

model = YOLO(model_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

for img_path in image_files:
    results = model.predict(img_path)
    img = cv2.imread(img_path)

    overlay = np.zeros_like(img)
    detection_results = []

    for result in results:
        masks = result.masks
    for mask, box in zip(masks, result.boxes):
        # Ensure mask is converted to numpy array and is on CPU
        mask_np = mask.numpy() if hasattr(mask, 'numpy') else mask.to('cpu', dtype=torch.uint8).numpy()
        
        # Convert boolean mask to uint8 if necessary
        if mask_np.dtype == np.bool_:
            mask_np = mask_np.astype(np.uint8) * 255  # Convert boolean mask to uint8

        polygons = mask_to_polygons(mask_np)

            random_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            points = np.array([polygon for polygon in polygons], dtype=np.int32) if polygons else []
            if points.size > 0:
                cv2.fillPoly(overlay, points, random_color)
            
            confidence = box.conf[0]
            if confidence > 0.4:
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                class_id = int(box.cls[0])
                class_name = model.names[class_id]

                detection_results.append({
                    "class_id": class_id,
                    "class_name": class_name,
                    "box": xyxy.tolist(),
                    "score": float(confidence),
                    "mask": polygons
                })

                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), random_color, 2)
                cv2.putText(img, f"{class_name} {confidence:.2f}", (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, random_color, 2)

    cv2.addWeighted(overlay, 0.4, img, 0.6, 0, img)
    unique_save_path = os.path.join(save_path, os.path.basename(img_path))
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")

    # Save results to JSON
    json_file_path = os.path.join(save_path, os.path.splitext(os.path.basename(img_path))[0] + ".json")
    with open(json_file_path, 'w') as json_file:
        json.dump({"file_name": os.path.basename(img_path), "instances": detection_results}, json_file, indent=4)



image 1/1 /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/123/resized/Model_1_0009_jpeg.rf.56cf23d9b2a62e6a83c0d3fa545f5d55_resize.jpg: 640x640 1 main_beam, 1 side_beam, 439.3ms
Speed: 1.3ms preprocess, 439.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)


AttributeError: 'Masks' object has no attribute 'dtype'. See valid attributes below.

    A class for storing and manipulating detection masks.

    Attributes:
        xy (list): A list of segments in pixel coordinates.
        xyn (list): A list of normalized segments.

    Methods:
        cpu(): Returns the masks tensor on CPU memory.
        numpy(): Returns the masks tensor as a numpy array.
        cuda(): Returns the masks tensor on GPU memory.
        to(device, dtype): Returns the masks tensor with the specified device and dtype.
    